In [ ]:
!pip install pymysql

In [ ]:
# product_list_전체신발_MFA.csv 파일을 MySQL tb_product 테이블에 삽입

import pandas as pd
import pymysql

# 1. CSV 불러오기
df = pd.read_csv("product_list_전체신발_MFA.csv")

# 2. 필요한 12개 컬럼만 추출
df = df[['id', 'brand', 'name', 'like_count', 'rating', 'discount',
         'price', 'gender', 'link', 'image', 'group', 'type']]

# 3. 컬럼명 DB 컬럼에 맞게 재설정
df.columns = ['pd_id', 'pd_brand', 'pd_name', 'pd_likes', 'pd_rating', 'pd_discount',
              'pd_price', 'pd_gender', 'pd_link', 'pd_image', 'pd_group', 'pd_type']

# 4. 전처리 함수 정의
def parse_likes(x):
    try:
        x = str(x).strip()
        if '만' in x:
            return int(float(x.replace('만', '')) * 10000)
        elif '천' in x:
            return int(float(x.replace('천', '')) * 1000)
        return int(x.replace(',', ''))
    except:
        return 0

def parse_price(x):
    try:
        return int(str(x).replace(',', '').replace('원', '').strip())
    except:
        return 0

def parse_rating(x):
    try:
        return float(str(x).strip())
    except:
        return 0.0

# 5. 전처리 적용
df['pd_likes'] = df['pd_likes'].apply(parse_likes)
df['pd_price'] = df['pd_price'].apply(parse_price)
df['pd_rating'] = df['pd_rating'].apply(parse_rating)

# 6. NaN 값을 None으로 변환 (MySQL 호환)
df = df.where(pd.notnull(df), None)

# 7. MySQL 연결
conn = pymysql.connect(
    host='project-db-campus.smhrd.com',
    port=3307,
    user='mueossa',
    password='12345',
    db='mueossa',
    charset='utf8mb4'
)
cursor = conn.cursor()

# 8. INSERT 쿼리 정의
insert_query = """
INSERT INTO tb_product (
    pd_id, pd_brand, pd_name, pd_likes, pd_rating, pd_discount,
    pd_price, pd_gender, pd_link, pd_image, pd_group, pd_type
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
    pd_brand=VALUES(pd_brand),
    pd_name=VALUES(pd_name),
    pd_likes=VALUES(pd_likes),
    pd_rating=VALUES(pd_rating),
    pd_discount=VALUES(pd_discount),
    pd_price=VALUES(pd_price),
    pd_gender=VALUES(pd_gender),
    pd_link=VALUES(pd_link),
    pd_image=VALUES(pd_image),
    pd_group=VALUES(pd_group),
    pd_type=VALUES(pd_type)
"""

# 9. 데이터 삽입
for _, row in df.iterrows():
    try:
        cursor.execute(insert_query, tuple(row))
    except Exception as e:
        print(f"Error inserting row {row['pd_id']}: {e}")

# 10. 완료
conn.commit()
cursor.close()
conn.close()
print("✅ 데이터 삽입 완료")


✅ 데이터 삽입 완료


In [ ]:
# 제품별_카테고리_비율.csv 파일을 MySQL tb_prod_feel_category 테이블에 삽입

import pandas as pd
import pymysql

# 1. CSV 불러오기
df = pd.read_csv("제품별_카테고리_비율.csv")

# 2. 컬럼명 매핑 (CSV → DB 컬럼명)
df.columns = [
    'pd_id', 'ct_comf', 'ct_fluffy', 'ct_light', 'ct_soft', 'ct_flat',
    'ct_pretty', 'ct_cute', 'ct_neat', 'ct_modern', 'ct_hip',
    'ct_wide', 'ct_standard', 'ct_narrow', 'ct_cost', 'ct_strong'
]

# 3. NaN → None (MySQL 호환)
df = df.where(pd.notnull(df), None)

# 4. MySQL 연결
conn = pymysql.connect(
    host='project-db-campus.smhrd.com',
    port=3307,
    user='mueossa',
    password='12345',
    db='mueossa',
    charset='utf8mb4'
)
cursor = conn.cursor()

# 5. INSERT 쿼리 정의
insert_query = """
INSERT INTO tb_prod_feel_category (
    pd_id, ct_comf, ct_fluffy, ct_light, ct_soft, ct_flat,
    ct_pretty, ct_cute, ct_neat, ct_modern, ct_hip,
    ct_wide, ct_standard, ct_narrow, ct_cost, ct_strong
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
    ct_comf=VALUES(ct_comf),
    ct_fluffy=VALUES(ct_fluffy),
    ct_light=VALUES(ct_light),
    ct_soft=VALUES(ct_soft),
    ct_flat=VALUES(ct_flat),
    ct_pretty=VALUES(ct_pretty),
    ct_cute=VALUES(ct_cute),
    ct_neat=VALUES(ct_neat),
    ct_modern=VALUES(ct_modern),
    ct_hip=VALUES(ct_hip),
    ct_wide=VALUES(ct_wide),
    ct_standard=VALUES(ct_standard),
    ct_narrow=VALUES(ct_narrow),
    ct_cost=VALUES(ct_cost),
    ct_strong=VALUES(ct_strong)
"""

# 6. 데이터 삽입
for _, row in df.iterrows():
    try:
        cursor.execute(insert_query, tuple(row))
    except Exception as e:
        print(f"Error inserting row {row['pd_id']}: {e}")

# 7. 완료
conn.commit()
cursor.close()
conn.close()
print("✅ tb_prod_feel_category 테이블 데이터 삽입 완료")


✅ tb_prod_feel_category 테이블 데이터 삽입 완료


In [ ]:
# 제품별_카테고리_상위25_여부_yn.csv 파일을 MySQL tb_prod_feel_yn 테이블에 삽입

import pandas as pd
import pymysql

# 1. CSV 불러오기
df = pd.read_csv("제품별_카테고리_상위25_여부_yn.csv")

# 2. 컬럼명 매핑 (CSV → DB 컬럼명)
df.columns = [
    'pd_id', 'ct_comf_yn', 'ct_fluffy_yn', 'ct_light_yn', 'ct_soft_yn', 'ct_flat_yn',
    'ct_pretty_yn', 'ct_cute_yn', 'ct_neat_yn', 'ct_modern_yn', 'ct_hip_yn',
    'ct_wide_yn', 'ct_standard_yn', 'ct_narrow_yn', 'ct_cost_yn', 'ct_strong_yn'
]

# 3. 소문자 'y', 'n'을 대문자로 변환 (DB 컬럼은 CHAR(1)로 'Y', 'N' 기대)
yn_columns = [col for col in df.columns if col.endswith('_yn')]
df[yn_columns] = df[yn_columns].applymap(lambda x: x.upper() if pd.notnull(x) else 'N')

# 4. NaN 값을 None으로 변환 (MySQL 호환)
df = df.where(pd.notnull(df), None)

# 5. MySQL 연결
conn = pymysql.connect(
    host='project-db-campus.smhrd.com',
    port=3307,
    user='mueossa',
    password='12345',
    db='mueossa',
    charset='utf8mb4'
)
cursor = conn.cursor()

# 6. INSERT 쿼리 정의
insert_query = """
INSERT INTO tb_prod_feel_yn (
    pd_id, ct_comf_yn, ct_fluffy_yn, ct_light_yn, ct_soft_yn, ct_flat_yn,
    ct_pretty_yn, ct_cute_yn, ct_neat_yn, ct_modern_yn, ct_hip_yn,
    ct_wide_yn, ct_standard_yn, ct_narrow_yn, ct_cost_yn, ct_strong_yn
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
    ct_comf_yn=VALUES(ct_comf_yn),
    ct_fluffy_yn=VALUES(ct_fluffy_yn),
    ct_light_yn=VALUES(ct_light_yn),
    ct_soft_yn=VALUES(ct_soft_yn),
    ct_flat_yn=VALUES(ct_flat_yn),
    ct_pretty_yn=VALUES(ct_pretty_yn),
    ct_cute_yn=VALUES(ct_cute_yn),
    ct_neat_yn=VALUES(ct_neat_yn),
    ct_modern_yn=VALUES(ct_modern_yn),
    ct_hip_yn=VALUES(ct_hip_yn),
    ct_wide_yn=VALUES(ct_wide_yn),
    ct_standard_yn=VALUES(ct_standard_yn),
    ct_narrow_yn=VALUES(ct_narrow_yn),
    ct_cost_yn=VALUES(ct_cost_yn),
    ct_strong_yn=VALUES(ct_strong_yn)
"""

# 7. 데이터 삽입
for _, row in df.iterrows():
    try:
        cursor.execute(insert_query, tuple(row))
    except Exception as e:
        print(f"Error inserting row {row['pd_id']}: {e}")

# 8. 완료
conn.commit()
cursor.close()
conn.close()
print("✅ tb_prod_feel_yn 테이블 데이터 삽입 완료")


✅ tb_prod_feel_yn 테이블 데이터 삽입 완료


In [ ]:
# product_list_전체신발_MFA.csv 파일에서 이미지 주소를 MySQL tb_prod_image 테이블에 삽입

import pandas as pd
import pymysql

# 1. CSV 파일 불러오기
df = pd.read_csv("product_list_전체신발_MFA.csv")

# 2. 상품 ID와 이미지 컬럼만 추출 (13번째 컬럼부터 끝까지)
df_images = df.iloc[:, [0] + list(range(12, df.shape[1]))]
df_images.columns = ['pd_id'] + [f'img_{i}' for i in range(1, df_images.shape[1])]

# 3. pd_id + 이미지 주소 한 줄씩 추출
img_data = []
for _, row in df_images.iterrows():
    pd_id = row['pd_id']
    for col in row.index[1:]:  # 이미지 컬럼만
        img_url = row[col]
        if pd.notnull(img_url):
            full_url = img_url.strip()
            if full_url.startswith("//"):
                full_url = "https:" + full_url
            img_data.append((int(pd_id), full_url))

# 4. MySQL 연결
conn = pymysql.connect(
    host='project-db-campus.smhrd.com',
    port=3307,
    user='mueossa',
    password='12345',
    db='mueossa',
    charset='utf8mb4'
)
cursor = conn.cursor()

# 5. INSERT 쿼리 정의
insert_query = """
INSERT INTO tb_prod_image (pd_id, img_name)
VALUES (%s, %s)
"""

# 6. 데이터 삽입
for data in img_data:
    try:
        cursor.execute(insert_query, data)
    except Exception as e:
        print(f"Error inserting image for product {data[0]}: {e}")

# 7. 완료
conn.commit()
cursor.close()
conn.close()
print("✅ tb_prod_image 테이블 이미지 삽입 완료")


✅ tb_prod_image 테이블 이미지 삽입 완료


In [ ]:
# product_sentiment.csv 파일을 MySQL tb_product 테이블에 감성 점수 업데이트
import pandas as pd
import pymysql

# 1. 감성 점수 파일 불러오기
product_sentiment = pd.read_csv("./data/product_sentiment.csv")

# 2. DB 연결
conn = pymysql.connect(
    host='project-db-campus.smhrd.com',
    port=3307,
    user='mueossa',
    password='12345',
    db='mueossa',
    charset='utf8mb4',
    autocommit=True
)

# 3. 감성 점수 업데이트
with conn.cursor() as cursor:
    for _, row in product_sentiment.iterrows():
        product_id = int(row["product_id"])  # ✅ int로 변환
        avg_sentiment_score = row["avg_sentiment_score"]

        update_query = """
        UPDATE tb_product
        SET sentiment = %s
        WHERE pd_id = %s
        """
        cursor.execute(update_query, (avg_sentiment_score, product_id))

print("✅ 감성 점수 업데이트 완료!")

conn.close()


✅ 감성 점수 업데이트 완료!
